<a href="https://colab.research.google.com/github/recanoy/Coursera_PythonForGenomicDataScience_FinalExam/blob/main/final_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Author: Raymart Jay E. Canoy
# Date: 11 October 2022
___

## (01) Installing Biopython

In [1]:
pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.0 MB/s 


## (02) Common Packages

In [2]:
import os
import sys
import subprocess
import requests
import Bio
from Bio import *
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT_ROOT_DIR = '/content/drive/MyDrive/Personal Files/(06) Genomic Data Science'

Mounted at /content/drive


## (03) Common Functions

In [4]:
# Installing packages that are not yet installed
def install(package):
  # This function installs packages that are not yet installed
  subprocess.call([sys.executable, "-m", "pip", "install", package])

# Extracting the file
def get_file(url, filename):
  # This function gets the file from the indicated URL
  # and saves it in the indicated filename
  req = requests.get(url)
  if req.status_code != 200: # the request is successful
    raise Exception("Could not get file")
  
  with open(filename, 'w') as fn:
    fn.write(req.text)

# Processing the file
def process_file(filename, filetype, info="all"):
  # This function processes the file contained in the indicated filename
  #
  # filename: Filename of the file containing the sequence
  # filetype: "fasta" or "genbank"
  # info: ["identifiers", "sequence", "length", "complete"]
  if info == "identifiers":
    return [seq_record.id for seq_record in SeqIO.parse(filename, filetype)]
  elif info == "sequence":
    return [seq_record.seq for seq_record in SeqIO.parse(filename, filetype)]
  elif info == "length":
    return [len(seq_record) for seq_record in SeqIO.parse(filename, filetype)]
  else:
    identifiers = [seq_record.id for seq_record in SeqIO.parse(filename, filetype)]
    sequence = [repr(seq_record.seq) for seq_record in SeqIO.parse(filename, filetype)]
    lengths = [len(seq_record) for seq_record in SeqIO.parse(filename, filetype)]

    complete_data = {
        "identifiers" : identifiers,
        "sequence" : sequence,
        "lengths" : lengths
    }

    return complete_data

# Determining the species
def species_finder(seq, E_VALUE_THRESH=0.01):
  # This functions finds the species of a inputted sequence
  #
  # Arguments:
  # seq: sequence
  # E_VALUE_THRESH: 0.01

  # Assessing whether biopython is installed
  # If not, it installs biopython and imports BLAST
  try:
    import Bio
    from Bio.Blast import NCBIWWW
    from Bio.Blast import NCBIXML
    print('Biopython is installed.')
  except ModuleNotFoundError:
    print('Biopython is not installed')
    install("biopython")

    # importing the biopython module
    import Bio
    from Bio.Blast import NCBIWWW
    from Bio.Blast import NCBIXML

  result_handle = NCBIWWW.qblast("blastn", "nt", seq)
  blast_record = NCBIXML.read(result_handle)

  for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
      if hsp.expect < E_VALUE_THRESH:
        print("*****Alignment*****")
        print("sequence:", alignment.title)
        print("length:", alignment.length)
        print("e value:", hsp.expect)
        print(hsp.query)
        print(hsp.match)
        print(hsp.sbjct)
        print("\n")


## (04) Getting the file

In [5]:
url = "https://d396qusza40orc.cloudfront.net/genpython/data_sets/dna.example.fasta"
filename = "dna.example.fasta"
filetype = "fasta"

get_file(url, filename)

## (05) Problem 1
* How many records are in the file?
  * A FASTA file is a single-line header, followed by lines of sequence data
  * The header line is distinguised from the sequence by the a greater-than (">") symbol in the first column.

In [6]:
print("There are %d record(s) in this FASTA file" % len(process_file(filename, filetype, info="identifiers")))

There are 25 record(s) in this FASTA file


## (06) Problem 2

* What are the lengths of the sequences in the file?
* What is the longest sequence?
* What is the sortest sequence?
* Is there more than one longest sequence?
* What are their identifiers?

In [7]:
# Determining the lengths of the sequences
def seq_len(lengths):
  # Determining the longest and shortest sequence lengths.
  max_seq = max(lengths) # maximum sequence length
  min_seq = min(lengths) # minimum sequence length
  
  max_ind = [ind for ind, value in enumerate(lengths) if value == max_seq]
  min_ind = [ind for ind, value in enumerate(lengths) if value == min_seq]

  num_max_seq = len([lengths[ind] for ind in max_ind])
  num_min_seq = len([lengths[ind] for ind in min_ind])

  print(
      """
      The maximum and minimum lengths of sequences is %d and %d respectively.
      There are %d record(s) which has the maximum length.
      There are %d record(s) which has the minimum length.
      """ % (max_seq, min_seq, num_max_seq, num_min_seq)
  )

  return max_ind, min_ind

In [8]:
lengths = process_file(filename, filetype, info="length")
identifiers = process_file(filename, filetype, info="identifiers")
max_ind, min_ind = seq_len(lengths)


      The maximum and minimum lengths of sequences is 4805 and 512 respectively.
      There are 1 record(s) which has the maximum length.
      There are 1 record(s) which has the minimum length.
      


In [9]:
for ind in max_ind:
  print("Identifier of sequence with max length: %s" % (identifiers[ind]))

Identifier of sequence with max length: gi|142022655|gb|EQ086233.1|323


In [10]:
for ind in min_ind:
  print("Identifier of sequence with min length: %s" % (identifiers[ind]))

Identifier of sequence with min length: gi|142022655|gb|EQ086233.1|521


## Problem 3

* A reading frame is a way dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons)
* There are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5')
* What is the length of the longest ORF in the file?
* What is the identifier of the sequence containing the longest ORF?
* For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier?
* What is the starting position of the longest ORF in the sequence that contains it? **The positions should indicate the character number in the sequence**

In [11]:
from Bio.Seq import Seq
my_seq = next(SeqIO.parse(filename, filetype)).seq

In [12]:
def extract_reading_frames(my_seq):
  # Extracting the reading the frames from a sequence
  reading_frame1 = []                   # Reading frame 1
  reading_frame2 = [Seq(my_seq[0])]     # Reading frame 2
  reading_frame3 = [Seq(my_seq[0:2])]   # Reading frame 3
  for frame in range(1, 4):
    if frame == 1:
      for i in range(0, len(my_seq), 3):
        reading_frame1.append(my_seq[i:i+3])
    elif frame == 2:
      for i in range(1, len(my_seq), 3):
        reading_frame2.append(my_seq[i:i+3])
    else:
      for i in range(2, len(my_seq), 3):
        reading_frame3.append(my_seq[i:i+3])

  return reading_frame1, reading_frame2, reading_frame3

In [13]:
def extract_ORF(read_frame):
  # Determining the ORFs in the reading frame
  # ORFs aresequences that begin with start codons and end with stop codons.
  # ORFs have the potential of coding for proteins.
  # start_codons: [Seq("ATG")]
  # stop_codons: [Seq("TAA"), Seq("TAG"), Seq("TGA")]
  # 
  # Argument(s):
  # read_frame: Either reading frame 1 or 2 or 3
  #
  # Output:
  # ORF_LIST: List of all the ORFs
  start_codons = [Seq("ATG")]                         # contains the list of start codons
  stop_codons = [Seq("TAA"), Seq("TAG"), Seq("TGA")]  # contains the list of stop codons

  start_ind = [ind for ind, value in enumerate(read_frame) if value == start_codons[0]]   # the indices containing the start codons
  stop_ind = [ind for ind, value in enumerate(read_frame) if (value == stop_codons[0] or value == stop_codons[1] or value == stop_codons[2])] # the indices containing the stop codons

  ORF_LIST = []                                       # empty list containing the ORFs
  ORF_IND = []                                        # empty list containing the indices of the ORF
  ORF = Seq("")                                       # empty ORF character

  if len(start_ind) > 0 and len(stop_ind) > 0:        # evaluate this section is there are start and stop codons in the sequence
    if start_ind[0] < stop_ind[0]:
      for ind in range(start_ind[0], stop_ind[0]):      # evaluating the element in the first element in the reading frame
        ORF += read_frame[ind]
      ORF_LIST.append(ORF)
      ORF_IND.append((start_ind[0], stop_ind[0]))

    for start_index in range(1, len(start_ind)):        # evaluating the second element in the start codon indices
      ORF = Seq("")                                          
      for stop_index in range(1, len(stop_ind)):        # evaluating the second element in the stop codon indices
        if ((stop_ind[stop_index] < start_ind[start_index]) or (start_ind[start_index] < stop_ind[stop_index-1])):
          continue
        
        else:
          ORF_IND.append((start_ind[start_index], stop_ind[stop_index]))
          for ind in range(start_ind[start_index], stop_ind[stop_index]+1): # 
            ORF += read_frame[ind]                     # appending the codons in the empty characters
          ORF_LIST.append(ORF)


  # Determining the lengths of the reading frames
  ORF_len = []
  if len(ORF_LIST) > 0:
    for i in range(0, len(ORF_LIST)):
      ORF_len.append(len(ORF_LIST[i]))
  
  # Determining the locations of the maximum codons
  max_ORF = []
  max_ind = []
  max_seq = []
  if len(ORF_len) > 0:
    max_ORF = [max(ORF_len)]
    max_ind = [ORF_IND[ind] for ind, value in enumerate(ORF_len) if value == max_ORF[0]]
    max_seq = [ORF_LIST[ind] for ind, value in enumerate(ORF_len) if value == max_ORF[0]]

  # Returning the results
  return ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq

In [14]:
read_frame1, read_frame2, read_frame3 = extract_reading_frames(my_seq)
print("Reading Frame 1:", read_frame1, "\n", "Reading Frame 2:", read_frame2, "\n", "Reading Frame 3:", read_frame3)

Reading Frame 1: [Seq('TCG'), Seq('GGC'), Seq('GAA'), Seq('GGC'), Seq('GGC'), Seq('AGC'), Seq('AAG'), Seq('TCG'), Seq('TCC'), Seq('ACG'), Seq('CGC'), Seq('AGC'), Seq('GCG'), Seq('GCA'), Seq('CCG'), Seq('CGG'), Seq('GCC'), Seq('TCT'), Seq('GCC'), Seq('GTG'), Seq('CGC'), Seq('TGC'), Seq('TTG'), Seq('GCC'), Seq('ATG'), Seq('GCC'), Seq('TCC'), Seq('AGC'), Seq('GCA'), Seq('CCG'), Seq('ATC'), Seq('GGA'), Seq('TCA'), Seq('AAG'), Seq('CCG'), Seq('CTG'), Seq('AAG'), Seq('CCT'), Seq('TCG'), Seq('CGC'), Seq('ATC'), Seq('AGG'), Seq('CGG'), Seq('CCA'), Seq('TAG'), Seq('TTG'), Seq('GCG'), Seq('CCA'), Seq('GTG'), Seq('ACC'), Seq('GTA'), Seq('CCA'), Seq('ACC'), Seq('GCC'), Seq('TTG'), Seq('ATG'), Seq('CGG'), Seq('CGC'), Seq('TCG'), Seq('GTC'), Seq('ATC'), Seq('GCT'), Seq('GCA'), Seq('TTG'), Seq('ATC'), Seq('GAG'), Seq('TAG'), Seq('CCA'), Seq('CCG'), Seq('CCG'), Seq('CCG'), Seq('CAA'), Seq('ATG'), Seq('CCC'), Seq('AGC'), Seq('ACG'), Seq('CCA'), Seq('ATG'), Seq('CGT'), Seq('TCT'), Seq('TCA'), Seq('TCC')

In [15]:
for frame in range(0, 3):
  if frame == 0:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame1)
    print("\n Processing reading frame 1 ...")
  elif frame == 1:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame2)
    print("\n Processing reading frame 2 ...")
  else:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame3)
    print("\n Processing reading frame 3 ...")
  print("ORF List:", ORF_LIST, "\n", "ORF Indices", ORF_IND, "\n", "ORF lengths", ORF_len, "\n", "Maximum Length of an ORF in a sequence", max_ORF, "\n", "Index of a the maximum ORF", max_ind, "\n", "Maximum sequence:", max_seq)


 Processing reading frame 1 ...
ORF List: [Seq('ATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGCCA'), Seq('ATGCGGCGCTCGGTCATCGCTGCATTGATCGAGTAG'), Seq('ATGCCCAGCACGCCAATGCGTTCTTCATCCACATAG'), Seq('ATGCGTTCTTCATCCACATAG'), Seq('ATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTT...TAA'), Seq('ATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCG...TAA'), Seq('ATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCT...TAA'), Seq('ATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACA...TAA'), Seq('ATGACAGCTCCGTTCAGGGGGAGGGGGTAA'), Seq('ATGGCACCTACATGGATCCCTCACTGCTTCCGTCTCTCGCGTGGTTCGCCCACG...TGA')] 
 ORF Indices [(24, 44), (55, 66), (72, 83), (77, 83), (122, 192), (151, 192), (152, 192), (167, 192), (183, 192), (230, 272)] 
 ORF lengths [60, 36, 36, 21, 213, 126, 123, 78, 30, 129] 
 Maximum Length of an ORF in a sequence [213] 
 Index of a the maximum ORF [(122, 192)] 
 Maximum sequence: [Seq('ATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTT...TAA')]

 Processing readi

## (04) Problem 4

* A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence
* For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and onece at position 3

In [16]:
my_seq = Seq("ACACACA")
n = 3

In [17]:
def repeats_finder(my_seq, n):
  repeats = {}
  # Starts at position 0
  rep = my_seq[0:0+n]

  count = 0
  position = []

  for j in range(0, len(my_seq)-n+1):
    print('Repeat Subject', rep, 'Repeat Query', my_seq[j:j+n])
    if rep == my_seq[j:j+n]:
      position.append(j+1)
      count += 1
  print('\n')

  if count > 1:
    repeats[rep] = [(count), position]

  # Starts at position 1
  for i in range(1, len(my_seq)):
    rep = my_seq[i:i+n]
    
    count = 0
    position = []
    if not key_finder(rep, repeats):
      for j in range(i, len(my_seq)-n+1):
        print('Repeat Subject', rep, 'Repeat Query', my_seq[j:j+n])
        if rep == my_seq[j:j+n]:
          position.append(j+1)
          count += 1
      print('\n')

      if count > 1:
        repeats[rep] = [(count), position]
    else:
      continue
  return(repeats)

def key_finder(rep, repeats):
  keys = repeats.keys()
  keys_list = [key[1] for key in enumerate(keys)]

  return any([key[1] == rep for key in enumerate(keys_list)])

In [18]:
reps = repeats_finder(my_seq, n)
print(reps)

Repeat Subject ACA Repeat Query ACA
Repeat Subject ACA Repeat Query CAC
Repeat Subject ACA Repeat Query ACA
Repeat Subject ACA Repeat Query CAC
Repeat Subject ACA Repeat Query ACA


Repeat Subject CAC Repeat Query CAC
Repeat Subject CAC Repeat Query ACA
Repeat Subject CAC Repeat Query CAC
Repeat Subject CAC Repeat Query ACA






{Seq('ACA'): [3, [1, 3, 5]], Seq('CAC'): [2, [2, 4]]}


## (05) Final Exam Questions

### 1: How many records are in the multi-FASTA file

In [19]:
fileurl = "https://d396qusza40orc.cloudfront.net/genpython/data_sets/dna2.fasta"
filename = "dna2.fasta"
filetype = "fasta"

In [20]:
get_file(fileurl, filename)

In [21]:
print("There are %d records in the multi-FASTA file." % len(process_file(filename, filetype, info="identifiers")))

There are 18 records in the multi-FASTA file.


### 2-3. What is the length of the longest sequence in the file?


In [21]:
lengths = process_file(filename, filetype, info="length")
max_ind, min_ind = seq_len(lengths)

print("\n Longest Sequence:", lengths[max_ind[0]], "\n", "\n Shortest Sequence", lengths[min_ind[0]])


      The maximum and minimum lengths of sequences is 4894 and 115 respectively.
      There are 1 record(s) which has the maximum length.
      There are 1 record(s) which has the minimum length.
      

 Longest Sequence: 4894 
 
 Shortest Sequence 115


### 4. What is the length of the longest ORF appeariing in reading frame 2 of any sequence

In [22]:
sequences = process_file(filename, filetype, info="sequence")

longest_ORF_rf2 = []
for seq_ind in range(0, len(sequences)):
  read_frame1, read_frame2, read_frame3 = extract_reading_frames(sequences[seq_ind])
  
  ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame2)
  longest_ORF_rf2.append(max_ORF)

print("The length of the longest ORF appearing in reading frame 2 of any sequences is", max(longest_ORF_rf2))

The length of the longest ORF appearing in reading frame 2 of any sequences is [1458]


### 5. What is the starting position of the longest ORF in reading frame 3 in any of the sequences?

In [23]:
## Determining the longest ORF 
longest_ORF_rf3 = []
longest_ORF_rf3_indices = []
longest_ORF_rf3_sequences = []
for seq_ind in range(0, len(sequences)):
  read_frame1, read_frame2, read_frame3 = extract_reading_frames(sequences[seq_ind])

  ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame3)
  longest_ORF_rf3.append(max_ORF)
  longest_ORF_rf3_sequences.append(max_seq)

max_ORF_rf3 = max(longest_ORF_rf3)
indices = [ind for ind, value in enumerate(longest_ORF_rf3) if value == max_ORF_rf3]

## Determining the position of the longest ORF in reading frame 3
longest_seq = sequences[indices[0]]
ORF = longest_ORF_rf3_sequences[indices[0]]
n = len(ORF[0])

start_position = []
for i in range(0, len(longest_seq)):
  if longest_seq[i:i+n] == ORF[0]:
    start_position.append(i)

print("The starting position of the longest ORF in reading frame 3 in any sequences is ", start_position[0]+1)

The starting position of the longest ORF in reading frame 3 in any sequences is  636


### (6) What is the length of the longest ORF appearing in any sequence and in any forward reading frame?

In [24]:
long_ORFs = {}

for frame in range(0, 3):
  if frame == 0:
    longest_ORF = []
    for seq_ind in range(0, len(sequences)):
      read_frame1, read_frame2, read_frame3 = extract_reading_frames(sequences[seq_ind])

      ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame1)
      longest_ORF.append(max_ORF)
    long_ORFs["Reading Frame 1"] = max(longest_ORF)
  elif frame == 1:
    longest_ORF = []
    for seq_ind in range(0, len(sequences)):
      read_frame1, read_frame2, read_frame3 = extract_reading_frames(sequences[seq_ind])

      ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame2)
      longest_ORF.append(max_ORF)                                                                    
    long_ORFs["Reading Frame 2"] = max(longest_ORF)
  else:
    longest_ORF = []
    for seq_ind in range(0, len(sequences)):
      read_frame1, read_frame2, read_frame3 = extract_reading_frames(sequences[seq_ind])

      ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame3)
      longest_ORF.append(max_ORF)
    long_ORFs["Reading Frame 3"] = max(longest_ORF)

print(long_ORFs)

{'Reading Frame 1': [2097], 'Reading Frame 2': [1458], 'Reading Frame 3': [1821]}


## (07) What is the length of the longest forward ORF that appears in the sequence with the identifier gi|142022655|gb|EQ086233.1|16?

In [25]:
identifiers = process_file(filename, filetype, info="identifiers")
identifier = "gi|142022655|gb|EQ086233.1|16"

ind = [identifiers.index(identifier)]
seq = sequences[ind[0]]

In [26]:
longest_ORF_myseq = {}
read_frame1, read_frame2, read_frame3 = extract_reading_frames(seq)

for frame in range(0, 3):

  if frame == 0:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame1)
    
    longest_ORF_myseq["Reading Frame 1:"] = max_ORF
  elif frame == 1:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame2)

    longest_ORF_myseq["Reading Frame 2:"] = max_ORF
  else:
    ORF_LIST, ORF_IND, ORF_len, max_ORF, max_ind, max_seq = extract_ORF(read_frame3)

    longest_ORF_myseq["Reading Frame 3:"] = max_ORF

print(longest_ORF_myseq)


{'Reading Frame 1:': [1509], 'Reading Frame 2:': [1458], 'Reading Frame 3:': [1644]}


### (08) Find the most frequently occuring repeat of length 6 in all sequences. How many times does it occur in all?

In [32]:
from joblib import load

def reps_compile(reps, reps_compiled):
  keys_compiled = reps_compiled.keys()   # keys in the compiled repeats
  vals_compiled = reps_compiled.values() # values in the compiled repeats

  keys = reps.keys()            # keys in the inputted repeats
  vals = reps.values()          # values in the inputted repeats

  keys_compiled_list = [key_comp[1] for key_comp in enumerate(keys_compiled)]
  keys_list = [key[1] for key in enumerate(keys)]

  if len(keys_list) > 0:
    for ind in range(0, len(keys_list)):
      if len(reps_compiled) > 0:
        if any([keys_list[ind] == val for i, val in enumerate(keys_compiled_list)]):
          reps_compiled[keys_list[ind]] += reps[keys_list[ind]][0]
      else:
        for ind in range(0, len(keys_list)):
          reps_compiled[keys_list[ind]] = reps[keys_list[ind]][0]
  
  return reps_compiled

from joblib import load

def load_data(path_):
  with open(path_, 'rb') as handle:
    dataset = load(handle)
  
  return(dataset)

In [ ]:
for l in [6, 12, 7]:
  reps_compiled = {}
  for seq in range(0, len(sequences)):
    reps = repeats_finder(sequences[seq], l)
    reps_compiled = reps_compile(reps, reps_compiled)

  if l == 6:
    with open(os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l06.pkl'), 'wb') as handle:
      pickle.dump(reps_compiled, handle)
  if l == 12:
    with open(os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l12.pkl'), 'wb') as handle:
      pickle.dump(reps_compiled, handle)
  else:
    with open(os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l07.pkl'), 'wb') as handle:
      pickle.dump(reps_compiled, handle)

In [37]:
filepath2 = os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l06.pkl')
dataset2 = load_data(filepath2)
print({k: v for k, v in sorted(dataset2.items(), key=lambda item: item[1], reverse=True)})

{Seq('GCGCGC'): 152, Seq('CGCGCG'): 151, Seq('GCCGCG'): 147, Seq('GCGCCG'): 135, Seq('CGCGGC'): 131, Seq('CGGCCG'): 123, Seq('CGCCGC'): 119, Seq('GCCGGC'): 117, Seq('GCGCGG'): 113, Seq('CGGCGC'): 107, Seq('GCGGCG'): 105, Seq('CCGCGC'): 104, Seq('CGCCGA'): 103, Seq('GCGACG'): 103, Seq('GCGCGA'): 101, Seq('CGCGCC'): 99, Seq('GGCGCG'): 98, Seq('CGCGAC'): 97, Seq('TCGCGC'): 95, Seq('CGACGC'): 95, Seq('CGGCGA'): 94, Seq('CGCGCA'): 92, Seq('CGACGA'): 92, Seq('TCGGCG'): 91, Seq('CCGGCG'): 89, Seq('TCGTCG'): 88, Seq('CGATCG'): 88, Seq('TGCGCG'): 87, Seq('TCGCCG'): 87, Seq('GCGGCC'): 86, Seq('GCCGCC'): 86, Seq('CGAGCG'): 86, Seq('GGCCGC'): 85, Seq('GCCGAC'): 81, Seq('GCGCTG'): 80, Seq('CGCCGG'): 80, Seq('CGCTCG'): 79, Seq('CGTCGC'): 79, Seq('GGCGGC'): 78, Seq('GCGATC'): 78, Seq('CCGCCG'): 78, Seq('ATCGCG'): 78, Seq('CGCGAT'): 77, Seq('CCGGCC'): 77, Seq('GCGTCG'): 76, Seq('CCGCGA'): 76, Seq('GATCGC'): 74, Seq('TCGACG'): 73, Seq('CCGACG'): 71, Seq('CGGCGT'): 70, Seq('GTCGCG'): 70, Seq('CGGCGG'): 

### (09) Find all repeats of length 12 in the input file. Let's use Max to specify the number of copies of the most frequent repeat of length 12.

* How many different 12-base sequences occur Max times?

In [38]:
filepath3 = os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l12.pkl')
dataset3 = load_data(filepath3)
print({k: v for k, v in sorted(dataset3.items(), key=lambda item: item[1], reverse=True)})

{Seq('TCGTCGGCGATC'): 2, Seq('CGCGGTTCCGCG'): 2, Seq('CTGCCGAACATC'): 2, Seq('TCGATCGCGTCG'): 2, Seq('CGATCGCGTCGG'): 2, Seq('CGACGCCGTCGA'): 2}


### (10) Which one of the following repeats of length 7 has a maximum number of occurrences?

In [40]:
filepath4 = os.path.join(PROJECT_ROOT_DIR, 'reps_compiled_l07.pkl')
dataset4 = load_data(filepath4)
print({k : v for k, v in sorted(dataset4.items(), key=lambda item:item[1], reverse=True)})

{Seq('CGCGCCG'): 59, Seq('CGCCGCG'): 58, Seq('GCCGCGC'): 58, Seq('GCGCGCG'): 58, Seq('GCGCGGC'): 56, Seq('CGCGGCG'): 55, Seq('CGCGACG'): 55, Seq('CGCGCGC'): 53, Seq('CGGCGCG'): 49, Seq('GCCGCCG'): 49, Seq('GCGGCCG'): 46, Seq('GCGCCGC'): 45, Seq('CGGCCGC'): 43, Seq('CGCGCGG'): 42, Seq('CGCGGCC'): 41, Seq('GGCGCGC'): 40, Seq('GCGCGCC'): 40, Seq('GCCGGCG'): 39, Seq('CGGCGGC'): 39, Seq('GATCGCG'): 39, Seq('GCTGCCG'): 38, Seq('CGCCGGC'): 37, Seq('GCCGGCC'): 37, Seq('CCGGCCG'): 37, Seq('CGTCGCG'): 37, Seq('CGACGGC'): 36, Seq('GCGCCGA'): 36, Seq('CGAGCGC'): 35, Seq('GCGATCG'): 34, Seq('CGACGCG'): 34, Seq('GCCGAGC'): 34, Seq('GCCGACG'): 34, Seq('CGCGATC'): 33, Seq('GCTCGCG'): 33, Seq('CGCGAGC'): 33, Seq('TGCGCGC'): 33, Seq('CGATCGC'): 33, Seq('TCGGCGA'): 32, Seq('CGGCACG'): 32, Seq('GCGACGC'): 32, Seq('GGCGATC'): 30, Seq('CTGCGCG'): 30, Seq('GCCGATC'): 30, Seq('CGGCGTG'): 29, Seq('CGTCGGC'): 29, Seq('GCGGCGG'): 29, Seq('CAGCGCG'): 29, Seq('CGCCGAC'): 29, Seq('GTGCGCG'): 29, Seq('GGCCGGC'): 28,